In [1]:
import sqlite3
import numpy as np

In [2]:
def fetchTrainingData(db, sqlCmd):
    try:
        conn = sqlite3.connect(db)
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
        result = cursor.fetchall()
        conn.commit()
        cursor.close()
        return result

    except sqlite3.Error as error:
        print("Failed to run sqlite command", error)
    finally:
        if (conn):
            conn.close()

In [3]:
db_path = 'db/categorised_swecris.sqlite'

In [4]:
for ix in range(2,18):
    sqlCmd = f'''
        SELECT abstract, Formas1 AS class FROM clean_swecris
            WHERE class == '{ix}' ORDER BY RANDOM() LIMIT 99
        '''
    result = np.array(fetchTrainingData(db_path, sqlCmd))
    if (ix == 2):
        training_data_Formas = np.copy(result)
    elif (result.size > 0):
        training_data_Formas = np.vstack((training_data_Formas, result))

In [16]:
init = False
for ix in range(2,18):
    sqlCmd = f'''
        SELECT abstract, Self1 AS class FROM clean_swecris 
            WHERE Self1 == '3' ORDER BY RANDOM()
            LIMIT
                CASE 
                    WHEN (99 - (SELECT count(Formas1) FROM clean_swecris WHERE Formas1 == '{ix}')) > 0
                        THEN (SELECT 99 - count(Formas1) FROM clean_swecris WHERE Formas1 == '{ix}')
                    ELSE 0
                END;
        '''
    result = np.array(fetchTrainingData(db_path, sqlCmd))
    if (not init and result.size > 0):
        training_data_Vinnova = np.copy(result)
        init = True
    elif (result.size > 0):
        training_data_Vinnova = np.vstack((training_data_Vinnova, result))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 2

In [6]:
for ix in range(2,17):
    sqlCmd = f'''
        SELECT abstract, goals AS class FROM clean_goals
            WHERE class == '{ix}';
        '''
    result = np.array(fetchTrainingData(db_path, sqlCmd))
    if (ix == 2):
        training_data_Goals = np.copy(result)
    else:
        training_data_Goals = np.vstack((training_data_Goals, result))

In [8]:
training_data = np.vstack((training_data_Goals, training_data_Formas, training_data_Vinnova))

In [12]:
import pandas as pd

In [13]:
training_data_df = pd.DataFrame(data=training_data[:,:], columns=['abstract', 'class'])

In [14]:
training_data_df

,abstract,class
0,ingen hung hung vara en främst dödsorsak i vär...,2
1,god hälsa och välbefinnande god hälsa vara en ...,3
2,god utbildning för alla utbildning vara en gru...,4
3,jämställdhet jämställdhet mellan kvinna och ma...,5
4,ren vatten och sanitet för alla vatten vara en...,6
...,...,...
2621,att under två år sända en forskare till inte...,17
2622,familj jurists webbverktyg underlätta för pr...,17
2623,ai förutspå ge stor möjlighet till en mycket...,17
2624,denna samla åtta svensk universitet och högs...,17
